## Import libraries and files

In [3]:
import pandas as pd
import warnings
import time
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

warnings.filterwarnings("ignore")
#pd.set_option('display.max_colwidth', -1)

In [4]:
def get_confusion_matrix_values(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    return(cm[0][0], cm[0][1], cm[1][0], cm[1][1])

In [10]:
#df_output = pd.read_csv("/Users/alaverre/Documents/Detecting_positive_selection/scripts/ML_optimisation/test_file.txt")
#df_output = pd.read_hdf(output_file_hdf, key="data")
path = "/Users/alaverre/Documents/Detecting_positive_selection/results/positive_selection/"
output_file_hdf = path + "human/CEBPA/Model/test_kmer_frequency.hdf5"
df_output = pd.read_hdf(output_file_hdf, key="data")

In [24]:
df_output

,ACTCAGGGTG,GCTGGAGTGA,AGGCAAAATG,AGCTTCTGGG,GAATGCTTCC,ACCAGTGGTG,AAATAAAACC,AGTTCATTTC,AACCCTGCTC,CATCCCCAGG,...,ACTGAAGGTG,CCGCAAACCG,GGGCATCGGA,CCTTTTCGAA,AACGCATTGA,ACGCGCCGAC,ACGTCCTCGC,CCGGTTTGAA,CTAAACTTGA,match
chr1_96443_96778_pos_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0.0,1
chr1_100279_100500_pos_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0.0,1
chr1_104934_105160_pos_3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0.0,1
chr1_136582_136775_pos_4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0.0,1
chr1_136877_137276_pos_5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
chr9_33939364_33939631_neg_920,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0.0,0
chr9_124230545_124230785_neg_921,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0.0,0
chr6_60735912_60736133_neg_922,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0.0,0
chr20_40737823_40738175_neg_923,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0.0,0


## Feature selection (find features that the most contribute to the prediction)

In [ ]:
#Correlation with output variable
cor_target = abs(df_output.corr()["match"])
#Selecting highly correlated features
relevant_features = list(pd.DataFrame(cor_target[cor_target>0.9]).index)

In [8]:
features = pd.DataFrame(cor_target[cor_target>0.9]).sort_values("match", ascending=False)

In [9]:
top_feature_list = list(features.head()[1:].index)
top_feature_list

['match_rating_comparison', 'token_set_ratio', 'w_ratio', 'token_sort_ratio']

## Find best classifiers for the prediction

In [10]:
#creating training and testing data
X = df_output[relevant_features[1:]].values
y = df_output['match'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, stratify=y)

In [13]:
#selecting and comparing state of the art classifiers to choose the best one
classifiers = {
    "DummyClassifier_stratified":DummyClassifier(strategy='stratified', random_state=0),    
    "KNeighborsClassifier":KNeighborsClassifier(3),
    "DecisionTreeClassifier":DecisionTreeClassifier(),
    "AdaBoostClassifier":AdaBoostClassifier(),
    "GradientBoostingClassifier":GradientBoostingClassifier(),
    "Perceptron": Perceptron(max_iter=40, eta0=0.1, random_state=1),
    "SupportVectorMachine":SVC(),
    "MLP": MLPClassifier(max_iter=100),
    "RandomForestClassifier":RandomForestClassifier(),
    "XGBClassifier":XGBClassifier(n_estimators=1000, learning_rate=0.1),
    "XGBClassifier finetuned":XGBClassifier(
                      scale_pos_weight=1,
                      learning_rate=0.01,  
                      colsample_bytree = 0.8,
                      subsample = 0.8,
                      objective='binary:logistic', 
                      n_estimators=1000, 
                      reg_alpha = 0.3,
                      max_depth=9, 
                      gamma=10)

}

df_results = pd.DataFrame(columns=['model', 'accuracy', 'mae', 'precision','recall','f1','roc','run_time','tp','fp','tn','fn'])
for key in classifiers:

    start_time = time.time()
    classifier = classifiers[key]
    model = classifier.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, zero_division=0)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    roc = roc_auc_score(y_test, y_pred)
    classification = classification_report(y_test, y_pred, zero_division=0)
    run_time = format(round((time.time() - start_time)/60,2))
    tp, fp, fn, tn = get_confusion_matrix_values(y_test, y_pred)

    row = {'model': key,
           'accuracy': accuracy,
           'mae': mae,
           'precision': precision,
           'recall': recall,
           'f1': f1,
           'roc': roc,
           'run_time': run_time,
           'tp': tp,
           'fp': fp,
           'tn': tn,
           'fn': fn,
          }
    df_results = df_results.append(row, ignore_index=True)

df_results.sort_values(['accuracy', 'precision'], ascending=[False, False]).reset_index(drop = True)

,model,accuracy,mae,precision,recall,f1,roc,run_time,tp,fp,tn,fn
0,DecisionTreeClassifier,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.0,148,0,152,0
1,AdaBoostClassifier,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.0,148,0,152,0
2,GradientBoostingClassifier,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.0,148,0,152,0
3,SupportVectorMachine,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.0,148,0,152,0
4,RandomForestClassifier,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.0,148,0,152,0
5,XGBClassifier,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.01,148,0,152,0
6,XGBClassifier finetuned,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.01,148,0,152,0
7,KNeighborsClassifier,0.996667,0.003333,1.000000,0.993421,0.996700,0.996711,0.0,148,0,151,1
8,MLP,0.996667,0.003333,0.993464,1.000000,0.996721,0.996622,0.0,147,1,152,0
9,Perceptron,0.993333,0.006667,1.000000,0.986842,0.993377,0.993421,0.0,148,0,150,2


,model,accuracy,mae,precision,recall,f1,roc,run_time,tp,fp,tn,fn
0,DecisionTreeClassifier,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.0,148,0,152,0
1,AdaBoostClassifier,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.0,148,0,152,0
2,GradientBoostingClassifier,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.0,148,0,152,0
3,SupportVectorMachine,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.0,148,0,152,0
4,RandomForestClassifier,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.0,148,0,152,0
5,XGBClassifier,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.01,148,0,152,0
6,XGBClassifier finetuned,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.01,148,0,152,0
7,KNeighborsClassifier,0.996667,0.003333,1.000000,0.993421,0.996700,0.996711,0.0,148,0,151,1
8,MLP,0.996667,0.003333,0.993464,1.000000,0.996721,0.996622,0.0,147,1,152,0
9,Perceptron,0.993333,0.006667,1.000000,0.986842,0.993377,0.993421,0.0,148,0,150,2
